# From Rest API to GraphQL 
Author: Zeiad Khamees  
June 29, 2022


## Goals 
* Build a simple web application that sorts different genres of music using Rest API and Flask. 
    * Set up  basic Flask web-server
    * Set up a database model using SCLAlchemy.
    * Work with HTTP methods and implement GET, POST, PUT and DELETE methods to adjust the database as we want.
* Implement the above using graphQL.  

#### __How the project relates to the Digital Engineering curriculum__
In the scope of this project, we are going to focus on the fourth principle ```Playing out of town``` from *Making Learning Whole* [Education at Bat: Seven Principles for Educators](https://www.gse.harvard.edu/news/uk/09/01/education-bat-seven-principles-educators). David perkins focuses on the the ability for learners to take their skills into the next level. He encourages learners to develpe transferrable knowledge that can be used flexibly in other fields. For instance, 
1. Students already know how to use Flask to develop a web-server. 
2. Students have worked with a different type of SQL to manipulate databases.
3. Students have a former knowledge of HTTP methods.

Considering the mentioned previous knowledge, students are encouraged to discover how to use this knowledge to achieve a simillar output with a different approach. By the end of this article, students will have:
1. An intoductory knowledge to a new language.
2. Learn the basic syntax of the language.
3. Learn how to query data from a database.


## Brief introduction
API stands for *Application Programming Interface*, A set of definitions and protocols for integrating web applications. In other words, it is a sutup that allows two softwares or applications to connect and communicate with each other. We know that every software differ form one another. When two softwares communicate, one software is going to deal with whatever in the backend which is the main server. The other software will make requests to the server to retrieve data in the frontend. Rest API stands for *Representational State Transfer*. As the name implies, this type of API helps transfer data over the internet. This is used to allow users to only view specific things from the backend. It is time that we introduce graphQL. graphQL is a query programming language for APIs and a server-side runtime. It simply gives the client the privelage of to ask for exactly what they need and only what the need, This makes it easier when dealing with more complex databases that contain huge amounts of data. Thus, unlike using a standart qurey language like SQL, graphQL is used to query data from any number of sources.Thankfully, both Rest APIs and graphQL pair with python nicely. In this project, we will try to build an application to store information about music genres using Rest API and then we will try to implement it with graphQL. This example is inspired from Caleb Curry where you can find it [here](https://www.youtube.com/watch?v=qbLc5a9jdXo&list=LL&index=3). 



### Setting up a basic Rest API
lets start by creating our oun Rest API with python. We will use Flask as our framework. Flask is highly recommended since it is a more simpler setup compared to other frameworks i.e. Django and Ruby. Flask runs on localhost:5000. However, any other framework will do the job. We are going to install the dependecies that we need to be able to work with the framework in our preferred environment. To do this, write the following in the terminal:

In [ ]:
# pip install flask
# pip install flask-sqlalchemy
# pip intall requests

: 

Create a file and name it _RestApp.py_ that we can write our initial code in. Start by intializing the flask app. For now, we will just return a simple string. This will give a response message of 200 OK, meaning the server is running without any problems. 

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return "Music Genres!"

if __name__ == "__main__":
    app.run()


: 

![Music Genres!](Screenshot_1.png) 

In the terminal, we would like to export our flask app to be able to run it. We have to do this every time we want to run our flask app.

In [ ]:
# SET FLASK_APP=RestApp.py
# SET FLASK_ENV=development
# flask run

: 

Now we are going to cretae a class to store music genres that inherits from an SQLAlchemy database model. The attribute unique here is used if we want every name to be different and the attribute nullable is used if we do not want any columns to be empty. repr method is used to represent the class's objects as strings that we can understand.

In [ ]:
class Music(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String, unique=True, nullable=False)
    description = db.Column(db.String(120))

    def __repr__(self):
        return f"{self.name} - {self.description}"

: 

### Link the database (SQLALchemy)

Going back to the terminal, we will find that the database is not yet defined. We want to create our data base using SQLAlchemy. To create a connection to the database, the database will be an instance of sqlite in the same directory. 

In [ ]:
from flask_sqlalchemy import SQLAlchemy

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///music.db'
db = SQLAlchemy(app)

: 

The next step is to import the database from our application to create our desired data tables in the terminal. This will create a music.db file in our repository. We will create our first entry in the database. Each piece of data is treated as an object of the class that we created. Thus, creating data is creating instances of the class Music and assigning it parameters to describe the data. Because we used the repr method, we can just type music and the list of music genres will be listed in a string format. Note that it is important to import the class as well in the fourth line or else the instance Music will not be defined. To test that we added a data row successfully we can simply type in music into the terminal and it will give us the data that we entered. 

In [ ]:
# python
# from RestApp import db
# db.create_all()
# from RestApp import Music
# db.session.add(music)
# db.session.commit()

: 

To add another music genre into the database, simply type the following:

In [ ]:
# db.session.add(Music(name="Rock", description="A type of pop music"))
# db.session.commit()
# Music.query.all()

: 

If we want to use our data with the server, we have to go back to the route(/music). Here we are going to implement a JSON list and iterate through the music genres. This way, the output will be in the form of a list of dictionaries containing the data. This way we can go back to the website and reload it, the data will be diplayed with one key which is __music__ and a pair of a list of dictionaries (i.e. name, description, etc.). 

In [ ]:
@app.route("/music")
def get_music():
    music = Music.query.all()

    output = []
    for music in Music:
        music_info = {'name': music.name, 'description': music.description}

        output.append(music_info)

: 

If we want to grap a specific genre, an easy way would be fetching it by its id. So to do this, we develop a new route. In this route, we will implement a new function and pass the id of a specific music genre. Because we are using dictionaries to store the data.   

![entries](Screenshot_2.png) 

In [ ]:
@app.route('/music/<id>')
def get_music_id(id):
    music = Music.query.get_or_404(id)         # this will either fetch the drink with a specific id or return a 404 error
    return {"name": music.name, "description":music.description}

: 

Lets test this by going back to the browser and type localhost5000/music/1.

![id](Screenshot_3.png) 

Now try to fetch a music genre with id 3. The result would be a 'Not Found' error as we have not yet entered a thired genre to our database.

![Error 404! ](Screenshot_4.png) 

##  Introduction to GraphQL
We introduced the query language GraphQL, but the question is what makes GraphQL more special and how does it differ from rest API's? REST API's are modeled as resources. We provide endpoints to access particular data and define which HTTP methods are used at each endpoint. On the other hand, GraphQL models the API as a graph.The GraphQL Schema Definition Language allows for defining the types and fields on each type the data. The types defined in a GraphQL schema are treated as nodes. The cleint can then make queries at a certain endpoint to access data at that particular node of interest. For more information, ypu can look at [The official GraphQl Documentaion](https://graphql.org/learn/). So to make it more simple, the main reasons the new declaritive query language is better are :
- Reduces web-app development efforts.
- Reduces bandwidth: querying only what we need and only what we need
- Reduces number of requests 

##### **Requirements**
To be able to implement our _RestApp.py_ example in GraphQL, we are going to need 3 requirements two of which we already installed in the previous example. The requirments are:
- Flask 
- Flask-SQLAlchemy 
- Ariadne 
- Graphene

As mentioned previously, Flask is just a framework that mekes it easier to build and configure web applicataions and Flask_SQLAlchemy is an extension used for flask that allows us to interact with our SQL database in python. Introducing two new libraries, `Ariadne` and `Graphene` are both libraries that help us implement GraphQL in python. 

So what is the difference between the two libraries?  


### Creating a new python virtual environment  
We will start building our new GraphQL server in a new module where we will name it _graphqlApp.py_ We need to create a new virtual environment to give us an isolation for the dependencies to be installed. We will use the same directory that we are working in with the RestApp, so we have to navigate to that directory. To navigate to the directory simply type the command `cd dir-name` with dir-name being the name of the directory you were originally working in. Create the virtual environment by running the following command:

In [ ]:

# python3 -m venv  graphApp

Next, activate the virtual environment. For windows,  simply type

In [ ]:
# graphApp\Scripts\activate.bat

If you are working with Linux or Mac, you may need to type the `source` command as follows:

In [ ]:
# source graphApp/bin/activate

### Writing our Schema
Now that we have activated our virtual environment, we want to create a new graphql file and name it _schema.graphql_ 